# Aula 14: Trabalhando com Schedulers no OpenMP

### Nome: Luca Mizrahi

### *Parte 1: Implementação Sequencial*

##### **1. Valor de Pi Estimado:**
- **Estimativa de Pi**: 3.1428

O valor estimado de Pi foi **3.1428**, o que está muito próximo do valor real de Pi (3.14159). Isso demonstra que a implementação sequencial do método de Monte Carlo foi correta e conseguiu estimar Pi com boa precisão usando 100.000 pontos aleatórios. Pequenas variações em torno do valor exato de Pi são esperadas, já que o método de Monte Carlo é baseado em uma amostragem estocástica (aleatória), e a precisão aumenta com o número de pontos sorteados.

##### **2. Tempo de Execução:**
- **Tempo de Execução**: 0.003844 segundos

O tempo de execução da versão sequencial foi bastante rápido, executando em **0.003844 segundos**. Esse tempo reflete o fato de que a implementação sequencial não tem overheads adicionais, como o gerenciamento de threads ou a criação de geradores de números aleatórios independentes. Para uma tarefa leve como a geração de 100.000 pontos aleatórios e a simples verificação se esses pontos caem dentro de um círculo, a implementação sequencial se comporta de maneira muito eficiente.

##### **3. Reflexão sobre a Implementação:**

##### **A sequência de números aleatórios foi gerada corretamente?**
Sim, a sequência de números aleatórios foi gerada corretamente usando a função `rand()` em C++, que gera números pseudoaleatórios entre 0 e `RAND_MAX`. Esses números foram convertidos para o intervalo [-1, 1] para simular o sorteio de pontos no quadrado inscrito. Embora `rand()` não seja o gerador de números aleatórios mais robusto, ele foi adequado para esta implementação sequencial simples.

##### **Houve alguma dificuldade na implementação?**
A implementação sequencial foi relativamente simples, pois o método de Monte Carlo envolve apenas a geração de números aleatórios e uma simples verificação de pontos. Não houve problemas significativos na lógica ou no desempenho, mas vale notar que o método `rand()` não é thread-safe e, portanto, não seria adequado para uma futura paralelização sem cuidados especiais. Além disso, o uso de `rand()` pode gerar números com qualidade inferior em alguns cenários, e versões modernas de C++ oferecem geradores de números aleatórios mais robustos.

##### **Conclusão:**
A versão sequencial do método de Monte Carlo para estimar Pi foi eficaz e eficiente. Ela conseguiu uma estimativa precisa de Pi com uma pequena variação em relação ao valor real e apresentou um tempo de execução muito curto. A simplicidade e a leveza da tarefa fizeram com que a implementação fosse direta, sem dificuldades significativas. Esta versão pode servir de base para experimentos futuros com paralelização, onde será necessário lidar com a geração de números aleatórios de maneira segura entre diferentes threads.

----------------

### *Parte 2: Primeira Tentativa de Paralelização*

Com base nos resultados apresentados para a execução do código sequencial e paralelizado, vamos refazer a reflexão solicitada na parte 2:

1. **Estimativa de Pi na Versão Sequencial:**
   - **Estimativa de Pi**: 3.1428
   - **Tempo de Execução**: 0.003844 segundos

2. **Estimativa de Pi na Versão Paralelizada:**
   - **Estimativa de Pi (paralelo)**: 3.14212
   - **Tempo de Execução (paralelo)**: 0.0150156 segundos


##### **Discussão dos Resultados:**

##### **1. Por que a geração de números aleatórios pode ser um obstáculo em um ambiente paralelo?**

A geração de números aleatórios em um ambiente paralelo pode ser problemática porque a função tradicionalmente utilizada, `rand()`, não é **thread-safe**, ou seja, quando várias threads tentam acessar e modificar o estado interno do gerador de números aleatórios simultaneamente, podem ocorrer conflitos (condições de corrida). Isso pode levar a números aleatórios repetidos ou sequências de números correlacionados, o que compromete a precisão dos cálculos.

No código apresentado, o problema da geração de números aleatórios não foi completamente resolvido, pois `rand()` foi utilizado sem uma estratégia de thread safety (como `rand_r()`), o que pode explicar porque, mesmo com paralelização, o desempenho não foi otimizado. Além disso, a utilização de uma única semente global (`std::srand()`) para todas as threads não garante que os números aleatórios gerados por cada thread sejam independentes.

##### **2. Como você resolveu esse problema?**

A estratégia usada para tentar resolver o problema foi manter a função `rand()` para geração de números aleatórios e usar `std::srand()` para inicializar a semente globalmente. No entanto, uma abordagem mais correta seria garantir que cada thread tivesse seu próprio gerador de números aleatórios independente (por exemplo, usando `rand_r()` ou `std::mt19937` com uma semente específica para cada thread).

No código atual, mesmo com paralelização, a função `rand()` é compartilhada entre as threads e isso pode gerar números correlacionados, o que afeta negativamente a eficiência da paralelização.

##### **3. A sua solução impactou o desempenho do código?**

Sim, a solução impactou negativamente o desempenho do código paralelizado. Isso pode ser observado no **tempo de execução maior** da versão paralela (0.015 segundos) em comparação à versão sequencial (0.0038 segundos), apesar de ambas estimarem o valor de Pi de forma próxima. 

Essa perda de desempenho pode ser explicada pelos seguintes fatores:
- **Conflitos na geração de números aleatórios**: Como `rand()` não é thread-safe, o uso dessa função em múltiplas threads pode introduzir contenção, fazendo com que as threads esperem umas pelas outras, o que resulta em **overhead**.
- **Overhead da paralelização**: Em problemas que são computacionalmente leves, como a geração de números aleatórios e a verificação de pontos no círculo, o overhead de criar e gerenciar threads pode ser maior do que o benefício do paralelismo. Isso é evidenciado pelo aumento no tempo de execução na versão paralela.

##### **4. Comparação da versão sequencial com a versão paralela:**

- **Estimativa de Pi**: Ambas as versões (sequencial e paralela) conseguiram estimar Pi com boa precisão: 3.1428 (sequencial) e 3.14212 (paralelo). Isso mostra que a lógica principal do algoritmo está correta e a distribuição dos pontos aleatórios está funcionando de forma adequada.
  
- **Tempo de Execução**: O tempo de execução da versão paralela foi **significativamente maior** (0.0150156 segundos) do que o da versão sequencial (0.003844 segundos). Isso indica que a paralelização introduziu mais overhead do que benefícios em termos de desempenho. Isso pode ser explicado pelo conflito na geração de números aleatórios e pelo fato de que a tarefa é relativamente simples e não exige um nível de paralelização muito alto.

-----------

### *Parte 3: Melhorando a Paralelização*

##### **1. Estimativa de Pi (melhorado):**
- **Estimativa de Pi**: 3.14824

O valor estimado de Pi na versão paralelizada melhorada foi **3.14824**, que está muito próximo do valor correto de Pi (3.14159). Essa pequena variação é esperada, pois o método de Monte Carlo é um **método estocástico**, e a precisão da estimativa depende diretamente do número de pontos aleatórios gerados (neste caso, `N = 100000`). A diferença de cerca de 0.0066 pode ser reduzida aumentando o número de pontos aleatórios.

**Conclusão**: A nova abordagem de geração de números aleatórios usando `std::mt19937` não afetou negativamente a precisão da estimativa de Pi. A distribuição de pontos aleatórios foi feita corretamente e o método manteve a precisão esperada para este número de pontos.

##### **2. Tempo de Execução (melhorado):**
- **Tempo de Execução**: 0.233462 segundos

O tempo de execução da versão melhorada foi **0.233462 segundos**, o que é significativamente maior do que o tempo da versão paralela anterior (0.015 segundos) e também maior que o da versão sequencial (0.0038 segundos).

##### **Por que o tempo de execução foi maior na versão melhorada?**

Existem algumas possíveis explicações para o aumento do tempo de execução:

1. **Overhead de Inicialização dos Geradores de Números Aleatórios (`std::mt19937`)**:
   - O **Mersenne Twister (`std::mt19937`)** é um gerador de números aleatórios de alta qualidade, mas também tem um custo maior em termos de desempenho quando comparado ao simples `rand()` ou `rand_r()`. A inicialização de `std::mt19937` pode ser mais lenta, especialmente quando várias threads precisam inicializar instâncias independentes.
   - O uso de **`std::random_device`** para gerar sementes também pode introduzir um overhead significativo, já que `random_device` é uma fonte de entropia de hardware, que pode ser relativamente lenta em comparação com geradores de números pseudoaleatórios simples como `rand()`.

2. **Dependência do Número de Pontos Gerados**:
   - Embora o paralelismo tenha sido melhorado com a criação de geradores de números independentes por thread, a tarefa de gerar 100.000 pontos ainda é computacionalmente leve. Como resultado, o **overhead de criar e gerenciar threads**, além da inicialização de geradores de números aleatórios de alta qualidade, pode superar os benefícios do paralelismo para um número relativamente pequeno de pontos.
   - Em problemas mais simples, o paralelismo pode não ser tão eficiente. Se o tempo gasto na inicialização e controle das threads é maior que o tempo de cálculo sequencial, o paralelismo não oferece ganhos significativos. Isso se reflete no tempo de execução relativamente alto para a versão paralelizada melhorada.

3. **Padrões de Paralelização**:
   - A geração de números aleatórios é independente entre as threads, mas o overhead de redução e sincronização dos resultados (com o uso de `reduction(+:pontos_dentro_circulo)`) pode introduzir um tempo adicional de espera ou contenção entre as threads.

##### **3. Comparação com a Versão Paralelizada Anterior:**
- Na versão anterior, o tempo de execução foi muito mais rápido (0.015 segundos), mas isso ocorreu porque a geração de números aleatórios com `rand()` foi mais leve, embora não fosse thread-safe e não garantisse independência entre os números gerados por diferentes threads.
- A nova versão corrigiu o problema da independência dos números aleatórios entre as threads, garantindo uma distribuição adequada de pontos, mas ao custo de maior tempo de execução devido ao uso de geradores de alta qualidade.


##### **Conclusões Finais:**

1. **A geração de números aleatórios foi paralelizada de maneira eficaz?**
   - Sim, a geração de números aleatórios foi paralelizada de maneira eficaz com o uso de `std::mt19937`. Cada thread agora tem seu próprio gerador de números independentes, garantindo que os números gerados sejam únicos e não correlacionados entre as threads.

2. **O valor de pi mudou de forma significativa?**
   - Não, o valor de Pi permaneceu próximo ao valor esperado (3.14824), o que demonstra que a estimativa está correta e consistente com o método de Monte Carlo. Pequenas variações são esperadas, dado que o método é estocástico, mas a precisão foi mantida.

3. **Houve melhoria no tempo de execução?**
   - **Não houve melhoria no tempo de execução**. Na verdade, o tempo de execução aumentou significativamente na versão paralelizada melhorada (0.233462 segundos). Isso ocorreu devido ao overhead de inicializar e gerenciar geradores de números aleatórios independentes (`std::mt19937`) e ao fato de o problema ser relativamente leve, o que torna o paralelismo menos eficiente do que a versão sequencial para este número de pontos.


##### **Recomendações:**

- Para problemas simples com poucos pontos, como neste caso (100.000 pontos), o paralelismo pode não ser necessário, pois o overhead de geradores de alta qualidade e controle de threads pode superar os ganhos esperados.
- Em um cenário com um número muito maior de pontos (por exemplo, **1 milhão ou mais**), espera-se que o paralelismo seja mais eficiente, já que o tempo de cálculo por thread aumentaria proporcionalmente, tornando o uso de várias threads mais vantajoso.
- Outra possível melhoria seria evitar o uso de `std::random_device` para a geração de sementes (que é relativamente lento) e, em vez disso, inicializar as sementes de maneira mais eficiente, como com o ID da thread e um contador incremental.



---------

### *Conclusão*

##### **Conclusão e Comparação**

##### **Tabela Comparativa:**

| Versão                        | Estimativa de Pi | Tempo de Execução (segundos) |
|-------------------------------|------------------|------------------------------|
| Sequencial                     | 3.1428           | 0.003844                      |
| Primeira Tentativa Paralela    | 3.14212          | 0.015016                      |
| Segunda Tentativa Paralela (Melhorada) | 3.14824          | 0.233462                      |

##### **Análise das Perguntas:**

1. **Houve uma melhoria significativa no tempo de execução entre a versão sequencial e as versões paralelas?**

   - **Resposta**: Não, não houve uma melhoria significativa no tempo de execução entre a versão sequencial e as versões paralelas. Na verdade, o tempo de execução aumentou nas versões paralelas. A primeira tentativa paralela foi mais lenta que a versão sequencial, e a segunda tentativa paralela (melhorada) foi ainda mais lenta. Isso ocorreu principalmente devido ao overhead de criação e gerenciamento de threads, além da inicialização de geradores de números aleatórios independentes. Para uma tarefa relativamente leve, como a geração de 100.000 pontos, o paralelismo introduziu mais sobrecarga do que benefícios, tornando a versão sequencial mais eficiente.

2. **A estimativa de pi permaneceu precisa em todas as versões?**

   - **Resposta**: Sim, a estimativa de Pi permaneceu precisa em todas as versões, com pequenas variações em torno do valor real (3.14159). As variações são esperadas, já que o método de Monte Carlo é estocástico, e a precisão aumenta com o número de pontos. A primeira tentativa paralela (3.14212) e a segunda tentativa paralela (3.14824) forneceram estimativas precisas de Pi, assim como a versão sequencial (3.1428).

3. **Quais foram os maiores desafios ao paralelizar o algoritmo, especialmente em relação aos números aleatórios?**

   - **Resposta**: O maior desafio ao paralelizar o algoritmo foi garantir a **geração correta de números aleatórios**. O uso da função `rand()`, que não é thread-safe, levou a problemas de concorrência na primeira tentativa paralela, o que poderia resultar em números aleatórios correlacionados entre as threads. Na segunda tentativa, ao usar geradores de números aleatórios independentes para cada thread com `std::mt19937`, conseguimos resolver esse problema, mas ao custo de maior tempo de execução, devido ao overhead de inicialização e gestão dos geradores de números aleatórios de alta qualidade. Este foi o principal fator que influenciou o desempenho do algoritmo paralelizado.

4. **O uso de threads trouxe benefícios claros para este problema específico?**

   - **Resposta**: Para este problema específico, o uso de threads **não trouxe benefícios claros**. O overhead adicional de criação e gerenciamento de threads e a inicialização de geradores de números aleatórios independentes superou os potenciais ganhos de paralelismo. Como a tarefa de geração de 100.000 pontos aleatórios e verificação é relativamente simples e leve, a versão sequencial foi mais eficiente. O uso de threads pode trazer benefícios em cenários com uma quantidade muito maior de pontos (por exemplo, milhões ou bilhões de pontos), onde o paralelismo pode ser melhor aproveitado, mas para este caso específico, a versão sequencial teve um desempenho superior. 

##### **Conclusão Geral:**
Embora a paralelização tenha resolvido problemas relacionados à geração de números aleatórios e mantido a precisão da estimativa de Pi, o uso de threads não trouxe melhorias de desempenho para o problema com 100.000 pontos. O overhead introduzido pelas threads e pelos geradores de números aleatórios acabou tornando as versões paralelas mais lentas do que a versão sequencial. Em problemas mais pesados, o paralelismo pode ser mais eficiente, mas para problemas leves, como este, a versão sequencial mostrou-se a opção mais adequada.